# Function Calling

## Install Packages

In [ ]:
%pip install azure-ai-projects==1.0.0b12
%pip install azure-identity
%pip install azure-ai-agents==1.1.0b3

## Import the libraries

In [14]:
import os, json, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import (
    FunctionTool,
    RequiredFunctionToolCall,
    SubmitToolOutputsAction,
    ToolOutput,
    ToolSet,
    MessageRole
)

## Create the AI Project Client

NOTE: 
- If you don't have Azure CLI, you first need to install it:
   curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash
- Login to Azure
   az login



In [15]:
# Create an Azure AI Client from an endpoint, copied from your Azure AI Foundry project.
# You need to login to Azure subscription via Azure CLI and set the environment variables
project_endpoint = os.environ["PROJECT_ENDPOINT"]  # Ensure the PROJECT_ENDPOINT environment variable is set

# Create an AIProjectClient instance
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),  # Use Azure Default Credential for authentication
)

## Define Functions and Function tool

In [16]:
def fetch_weather(location: str) -> str:
    """
    Fetches the weather information for the specified location.

    :param location (str): The location to fetch weather for.
    :return: Weather information as a JSON string.
    :rtype: str
    """
    # In a real-world scenario, you'd integrate with a weather API.
    # Here, we'll mock the response.
    mock_weather_data = {
        "New York": "Sunny, 25°C", 
        "London": "Cloudy, 18°C", 
        "Tokyo": "Rainy, 22°C"
    }
    weather = mock_weather_data.get(location, "Weather data not available for this location.")
    weather_json = json.dumps({"weather": weather})
    return weather_json

def fetch_restaurant(location: str) -> str:
    """
    Fetches the restaurant information for the specified location.

    :param location (str): The location to fetch the restaurant for.
    :return: Restaurant information as a JSON string.
    :rtype: str
    """
    # In a real-world scenario, you'd integrate with a restaurant API.
    # Here, we'll mock the response.
    mock_restaurant_data = {
        "New York": "Tatiana by Kwame Onwuachi, Katz’s Delicatessen, Peter Luger Steakhouse, Sylvia's, Nathan's Famous", 
        "London": "St. JOHN, Señor Ceviche, Gloria and Circolo Popolare, Normah's, Bouchon Racine", 
        "Tokyo": "Chanko & Wanko Restaurant Asakusa Sumo Club, Sky Restaurant 634 Musashi, Ichiran, Shibuya, Rokkasen Otakibashiidori, Hakushu - Kobe Teppanyaki"
    }
    restaurant = mock_restaurant_data.get(location, "Restaurant data not available for this location.")
    restaurant_json = json.dumps({"restaurant": restaurant})
    return restaurant_json

def fetch_budget() -> str:
    """
    Fetches the budget information for the specified location.
    :return: budget information as a JSON string.
    :rtype: str
    """
    # In a real-world scenario, you'd integrate with a another API.
    # Here, we'll mock the response.
    mock_budget_data = {
        "New York": """
            Budget Travelers: Around $121 per day. This includes staying in hostels, eating at budget restaurants, and using public transportation.
            Mid-Range Travelers: Approximately $324 per day. This covers mid-range hotels, dining at average restaurants, and some paid attractions.
            Luxury Travelers: About $923 per day. This includes luxury hotels, fine dining, and private transportation.
        """, 
        "London": """
            Budget Travelers: Around $75 per day. This includes staying in hostels, cooking your own meals, and using public transport.
            Mid-Range Travelers: Approximately $195 per day. This covers mid-range hotels, dining at average restaurants, and some paid attractions.
            Luxury Travelers: About $517 per day. This includes luxury hotels, fine dining, and private transportation.
        """, 
        "Tokyo": """
            Budget Travelers: Around $100 per day. This includes staying in hostels, eating at budget restaurants, and using public transportation.
            Mid-Range Travelers: Approximately $286 per day. This covers mid-range hotels, dining at average restaurants, and some paid attractions.
            Luxury Travelers: About $908 per day. This includes luxury hotels, fine dining, and private transportation.
        """
    }
    budget_json = json.dumps({"budget": mock_budget_data})
    return budget_json

# Define user functions
user_functions = {fetch_weather, fetch_restaurant, fetch_budget}

# Initialize the FunctionTool with user-defined functions
functions = FunctionTool(functions=user_functions)
toolset = ToolSet()
toolset.add(functions)

# To enable tool calls executed automatically
project_client.agents.enable_auto_function_calls(toolset)

## Create the agent

In [17]:
# Create an agent with the file search tool
agent = project_client.agents.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],  # Model deployment name
    name="my-functioncalling-agent",  # Name of the agent
    instructions="You are a helpful agent and can search information from provided tools",  # Instructions for the agent
    toolset=toolset,  # Toolset containing all the tools
  
)
print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_FXtw6LpSMf8byQY3mEV3l9rd


## Create thread

In [18]:
# Create a thread for communication
thread = project_client.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_wcbRQK40m0ROpYSArqV4QEDI


## Helper function
- adds messages to the thread
- run the agent
- display the agent response

In [19]:
def run_agent(user_input):
    # Add a message to the thread
    message = project_client.agents.messages.create(
        thread_id=thread.id,
        role="user",  # Role of the message sender
        content=user_input,  # Message content
    )
    print(f"Created message, ID: {message['id']}")

     # Create and process agent run in thread with tools
    run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    # Check the status of the run and print the result
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")
    elif run.status == "completed":
        last_msg = project_client.agents.messages.get_last_message_text_by_role(thread_id=thread.id, role=MessageRole.AGENT)
        if last_msg:
            print(f"Agent Response: {last_msg.text.value}")

In [20]:
def run_agent_details(user_input):
    # Add a message to the thread
    message = project_client.agents.messages.create(
        thread_id=thread.id,
        role="user",  # Role of the message sender
        content=user_input,  # Message content
    )
    print(f"Created message, ID: {message['id']}")

    # Create a run for the agent
    run = project_client.agents.runs.create(thread_id=thread.id, agent_id=agent.id)
    print(f"Created run, ID: {run.id}")

    # Poll the run status until it completes or fails
    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = project_client.agents.runs.get(thread_id=thread.id, run_id=run.id)

        # Check if the run requires action
        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolOutputsAction):

            tool_calls = run.required_action.submit_tool_outputs.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                project_client.agents.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_outputs = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredFunctionToolCall):
                    try:
                        print(f"Executing tool call: {tool_call}")
                        output = functions.execute(tool_call)
                        tool_outputs.append(
                            ToolOutput(
                                tool_call_id=tool_call.id,
                                output=output,
                            )
                        )
                    except Exception as e:
                        print(f"Error executing tool_call {tool_call.id}: {e}")

            print(f"Tool outputs: {tool_outputs}")
            if tool_outputs:
                project_client.agents.runs.submit_tool_outputs(thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs)

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")

    # Check the status of the run and print the result
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")
    elif run.status == "completed":
        last_msg = project_client.agents.messages.get_last_message_text_by_role(thread_id=thread.id, role=MessageRole.AGENT)
        if last_msg:
            print(f"Agent Response: {last_msg.text.value}")

## Run the agent

In [21]:
user_input = "What is the weather in New York?"
run_agent(user_input)

Created message, ID: msg_tTlxQusrmueNTAweEvt4N6kt
Run finished with status: completed
Agent Response: The weather in New York is currently sunny with a temperature of 25°C.


In [22]:
user_input = "What can I eat in London?"
run_agent_details(user_input)

Created message, ID: msg_04Ffxl10Dcycr2zvHisFQLq2
Created run, ID: run_ngguxyMI92iiWGiiy32XdZNa
Executing tool call: {'id': 'call_JfGqhgKZ5LyRALHYs8Ru0xEW', 'type': 'function', 'function': {'name': 'fetch_restaurant', 'arguments': '{"location":"London"}'}}
Tool outputs: [{'tool_call_id': 'call_JfGqhgKZ5LyRALHYs8Ru0xEW', 'output': '{"restaurant": "St. JOHN, Se\\u00f1or Ceviche, Gloria and Circolo Popolare, Normah\'s, Bouchon Racine"}'}]
Current run status: requires_action
Current run status: in_progress
Current run status: in_progress
Current run status: in_progress
Current run status: completed
Run completed with status: completed
Agent Response: In London, you have several great dining options including:

- St. JOHN
- Señor Ceviche
- Gloria and Circolo Popolare
- Normah's
- Bouchon Racine


In [23]:
user_input = "With a small budget of 300USD, where can I go for 4 days?"
run_agent_details(user_input)

Created message, ID: msg_3tbOlCpyuS1knigGwwkFuEVm
Created run, ID: run_X1fPUDwJUbH76vIzvGb5a7qT
Executing tool call: {'id': 'call_cM7iPO0KMZqSI6O8IA7CKqpM', 'type': 'function', 'function': {'name': 'fetch_budget', 'arguments': '{}'}}
Tool outputs: [{'tool_call_id': 'call_cM7iPO0KMZqSI6O8IA7CKqpM', 'output': '{"budget": {"New York": "\\n            Budget Travelers: Around $121 per day. This includes staying in hostels, eating at budget restaurants, and using public transportation.\\n            Mid-Range Travelers: Approximately $324 per day. This covers mid-range hotels, dining at average restaurants, and some paid attractions.\\n            Luxury Travelers: About $923 per day. This includes luxury hotels, fine dining, and private transportation.\\n        ", "London": "\\n            Budget Travelers: Around $75 per day. This includes staying in hostels, cooking your own meals, and using public transport.\\n            Mid-Range Travelers: Approximately $195 per day. This covers mid

## Cleanup resources

In [24]:
# Cleanup resources

# Delete the agent after use
project_client.agents.delete_agent(agent.id)
print("Deleted agent")


Deleted agent
